In [12]:
%pip install matplotlib
%pip install darts
%pip install catboost


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
from datetime import datetime
from darts import TimeSeries
from darts.models import CatBoostModel
import pandas as pd
import os
import kaggle_metric
import utils

In [14]:
df = utils.create_df('./data/kernel/receivals.csv')
df.head()

,date_arrival,rm_id,net_weight
0,2004-06-15,342,0.0
1,2004-06-16,342,0.0
2,2004-06-17,342,0.0
3,2004-06-18,342,0.0
4,2004-06-19,342,0.0


In [15]:
series_lst = TimeSeries.from_group_dataframe(df, 'rm_id', 'date_arrival','net_weight')

In [16]:
series_dict = {}
train_dict = {}
val_dict = {}
for i in range(len(series_lst)):
    train_i, val_i = series_lst[i].split_after(0.8)
    
    rm_id = series_lst[i].static_covariates["rm_id"] if "rm_id" in series_lst[i].static_covariates else None
    series_dict[rm_id.values[0]] = series_lst[i]
    train_dict[rm_id.values[0]] = train_i
    val_dict[rm_id.values[0]] = val_i

In [17]:
model_dict = {}
loss = []

lst = []
for key in train_dict:
    print(f"Training model for rm_id: {key}")
    train_values = train_dict[key].values()
    model = CatBoostModel(lags=30, use_static_covariates=False, random_state=42)
    try:
        model.fit(train_dict[key])
        model_dict[key] = model
        out = model_dict[key].predict(len(val_dict[key]), series=train_dict[key])
        actual_steps = out.values().clip(min=0)
        actual_steps = actual_steps.cumsum()
        actual_steps[actual_steps < 100] = 0

        # Store predictions and actual values in a dataframe
        df_eval = pd.DataFrame({
            'predicted_weight': actual_steps.flatten(),
            'weight': val_dict[key].cumsum().values().flatten()
        })

    except Exception as e:
        df_eval = pd.DataFrame({
            'predicted_weight': [0]*len(val_dict[key]),
            'weight': val_dict[key].cumsum().values().flatten()
        })
        print(f"Error training model for rm_id {key}: {e}")

    lst.append(df_eval)

df_eval = pd.concat(lst).reset_index(drop=True)
df_eval['ID'] = df_eval.index
score = kaggle_metric.score(df_eval[['ID', 'weight']], df_eval[['ID', 'predicted_weight']])
print(f"Validation score: {score}")
    


Training model for rm_id: 342.0
Error training model for rm_id 342.0: catboost/libs/metrics/metric.cpp:6935: All train targets are equal
Training model for rm_id: 343.0
Training model for rm_id: 345.0
Training model for rm_id: 346.0
Training model for rm_id: 347.0
Training model for rm_id: 348.0
Training model for rm_id: 353.0
Training model for rm_id: 354.0
Training model for rm_id: 355.0
Training model for rm_id: 357.0
Training model for rm_id: 358.0
Training model for rm_id: 360.0
Training model for rm_id: 362.0
Training model for rm_id: 364.0
Training model for rm_id: 365.0
Training model for rm_id: 366.0
Training model for rm_id: 367.0
Training model for rm_id: 368.0
Training model for rm_id: 369.0
Training model for rm_id: 374.0
Training model for rm_id: 375.0
Training model for rm_id: 378.0
Training model for rm_id: 379.0
Training model for rm_id: 380.0
Training model for rm_id: 381.0
Training model for rm_id: 383.0
Training model for rm_id: 386.0
Training model for rm_id: 387.0

In [18]:
df_eval[df_eval['weight'] > 0]

,predicted_weight,weight,ID
101094,310.079657,1152.0,101094
101095,310.666929,1152.0,101095
101096,311.254201,1152.0,101096
101097,311.841473,1152.0,101097
101098,312.428745,1152.0,101098
...,...,...,...
304698,0.000000,48180.0,304698
304699,0.000000,48180.0,304699
304700,0.000000,48180.0,304700
304701,0.000000,48180.0,304701


In [19]:
# prediction_mapping = pd.read_csv('./data/prediction_mapping.csv')
# prediction_mapping['forecast_start_date'] = pd.to_datetime(prediction_mapping['forecast_start_date'], utc=True).dt.tz_localize(None).dt.normalize()
# prediction_mapping['forecast_end_date'] = pd.to_datetime(prediction_mapping['forecast_end_date'], utc=True).dt.tz_localize(None).dt.normalize()
# prediction_mapping = prediction_mapping.sort_values(['rm_id', 'forecast_end_date'])
# max_date = pd.Timestamp('2025-05-31')
# lst = []
# for rm_id, group in prediction_mapping.groupby('rm_id'):
#     print(f"Processing rm_id {rm_id}")
#     if rm_id not in model_dict:
#         print(f"No model for rm_id {rm_id}, setting predicted_weight to 0")
#         group['predicted_weight'] = 0
#     else:
    
#         last_train = series_dict[rm_id].time_index[-1]
#         n_steps = (max_date - last_train).days - 1
#         print(f"Predicting {n_steps} steps for rm_id {rm_id} because last known date is {last_train}")
#         preds = model_dict[rm_id].predict(n_steps, series=series_dict[rm_id])
#         actual_steps = preds.values().clip(min=0)
#         actual_steps = actual_steps.cumsum()
#         # group['predicted_weight'] = actual_steps
#         actual_steps[actual_steps < 100] = 0
#         group['predicted_weight'] = actual_steps
#     lst.append(group)

# df_final = pd.concat(lst)

In [20]:
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# submission_path = f'./data/submission/submission_{timestamp}.csv'
# if not os.path.exists('./data/submission/'):
#     os.makedirs('./data/submission/')
    

# df_final[['ID', 'predicted_weight']].to_csv(submission_path, index=False)